In [14]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import random 
import copy

# Projeto: O Desafio NetFlix
Outros datasets: https://github.com/caserec/Datasets-for-Recommender-Systems

Um problema que existe hoje em dia com as grandes empresas de streaming (Netflix, Spotify, etc.) é que elas têm um acervo de conteúdo muito grande, e os usuários tendem a gostar, cada um, de uma pequena parte desse acervo. Então, como poderíamos escolher quais filmes vão aparecer tela inicial do seu streaming?

O objetivo deste projeto é fazer um sistema que toma essa decisão.

## Quais dados temos à disposição?

Neste projeto, trabalharemos com o [The Movies Dataset](https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset), que tem, entre outras coisas, a avaliação de usuários em relação a filmes. Essa avaliação está na tabela `ratings.csv` - mas, opcionalmente, pode ser usada a `ratings_small.csv`, que tem somente um subconjunto desses dados.

In [6]:
import pandas as pd
df = pd.read_csv('ratings_small.csv')
df.head(2)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179


In [ ]:
df["movie_id"] # filmes
df["userId"] # usuários
df["rating"] # ratings

Veja que esse dataframe pode facilmente ser transformado numa matriz $A$ que tem uma linha para cada usuário (identificado por `userId`) e uma coluna para cada filme (identificado por `movieId`). O conteúdo da matriz é o *rating* que o usuário atribuiu ao filme. Podemos ignorar a coluna `timestamp`.


In [7]:
df_matrix = df.to_numpy()
df_matrix

array([[1.00000000e+00, 3.10000000e+01, 2.50000000e+00, 1.26075914e+09],
       [1.00000000e+00, 1.02900000e+03, 3.00000000e+00, 1.26075918e+09],
       [1.00000000e+00, 1.06100000e+03, 3.00000000e+00, 1.26075918e+09],
       ...,
       [6.71000000e+02, 6.36500000e+03, 4.00000000e+00, 1.07094036e+09],
       [6.71000000e+02, 6.38500000e+03, 2.50000000e+00, 1.07097966e+09],
       [6.71000000e+02, 6.56500000e+03, 3.50000000e+00, 1.07478472e+09]])


## Desafio: eu vou gostar deste filme?

O que gostaríamos de saber é qual nota um usuário deveria atribuir a um filme que ele ainda não assistiu. Para isso, o procedimento será o seguinte.

1. Vamos escolher aleatoriamente um dos elementos da matriz $A$ e atribuir a ele um valor aleatório, gerando a matriz $B$.
1. O sistema receberá como entrada a matriz $B$ e a posição $i,j$ do valor aleatório. Neste momento, ele não teve acesso à matriz $A$, e, portanto, não tem como saber qual é o valor "real".
1. O sistema deverá retornar o valor real que estava na matriz $A$.
1. Esse procedimento deverá ser repetido várias vezes, de forma a gerar um histograma dos erros cometidos.


In [15]:
# ZOAR COM A MATRIZ
def randomiza_item_matriz(A):
    B = copy.deepcopy(A)
    r_i = random.randint(0,len(A))
    r_j = random.randint(0,len(A[0]))
    r_v = random.randint(0,len(B))
    B[r_i,r_j] = r_v
    return B, (r_i,r_j)

E, ij = randomiza_item_matriz(df_matrix)

In [16]:
print(df_matrix [ij[0], ij[1]])
print(E [ij[0], ij[1]])


382.0
9611.0



## Como o sistema funciona?

A ideia do sistema de recomendação é que existem "perfis" típicos de usuários. Os perfis, para este problema, são vetores que mostram que notas são tipicamente atribuídas para cada filme por usuários daquele perfil. Por exemplo, talvez tenhamos dois perfis e três filmes, e nesse caso poderíamos ter os perfis:

* $p_0 = [2, 5, 2]$, isto é, o perfil $0$ é de uma pessoa que gosta muito do filme $f_1$, e
* $p_1 = [5, 0, 4]$, isto é, o perfil $1$ é de uma pessoa que gosta dos filmes $f_0$ e $f_2$. 

Porém, sabemos que usuários reais raramente se comportam estritamente como um perfil. As notas realmente atribuídas por um usuário aos filmes, então, são modeladas como combinações lineares dos perfis. Por exemplo, podemos ter usuários:

* $u_0 = 0.1 p_0 + 0.9 p_1$, para um usuário muito próximo de $p_1$ mas distante de $p_0$,
* $u_1 = 0.1 p_0 + 0.1 p_1$, para um usuário distante tanto de $p_0$ quanto de $p_1$,

e assim por diante.

Então, o que precisamos é de uma maneira de mapear usuários para perfis, e então perfis para filmes. Precisamos então *decompor* nossa matriz $A$ de usuários $\times$ filmes em componentes:

$
A = X Y Z,
$

onde:
* $A$ tem uma linha por usuário e uma coluna por filme.

In [ ]:
# Fazer Matriz

* $X$ tem uma linha por usuário e uma coluna por perfil,

In [ ]:
# Fazer Matriz

* $Y$ é quadrada e mapeia perfis para perfis,

In [ ]:
# Fazer Matriz

* $Z$ tem uma linha por perfil e uma coluna por filme.

In [ ]:
# Fazer Matriz


Isso se parece bastante com algo que já fizemos nesta aula!

Em nosso teste, ao aleatorizarmos um elemento da matriz $A$, estamos inserindo ruído. Como poderíamos remover esse ruído?

## Descrição do projeto

Neste projeto, o grupo deverá fazer um sistema preditor de nota de filmes por usuário, que funciona nas condições que foram citadas no enunciado (temos conhecimento de todo o dataset, exceto do par filme-usuário específico). O sistema projetado deve ser avaliado usando um histograma dos erros ao longo de várias estimativas. O número de estimações deve ser, no mínimo, mil. O projeto deve ser colocado em um repositório GitHub específico.

Anotações importantes:

1. O grupo deve enviar um link para o repositório GitHub onde está localizada a biblioteca.
2. No diretório principal do repositório, deve haver um programa `demo.py`, que, quando executado, executa todos os testes que geram o histograma de resultados.
3. Como o objetivo do projeto é exatamente implementar o sistema de recomendação, não é permitido usar bibliotecas que fazem recomendações. Toda a parte de algoritmos e álgebra linear deve ser feita pelo próprio grupo usando Numpy ou Scipy.

**ENTREGAS**
* Link para o repositório onde está a biblioteca.
* No `README.md` do repositório, deve haver uma discussão sobre como o sistema funciona. Essa discussão deve corresponder ao que foi feito no código.
* Inclua também, no próprio `README.md`, instruções sobre como rodar o `demo.py` e como usar suas funcionalidades.
* O `README.md` também deve ter uma discussão dos resultados encontrados, incluindo o histograma dos erros e uma conclusão, baseada em dados, sobre se o grupo acredita que o sistema proposto poderia ser usado em produção.

**RUBRICA**

O projeto será avaliado usando a rubrica abaixo. Os níveis são cumulativos, isto é, para passar de um nível, *todos* os requisitos dele devem ser cumpridos. As rubricas foram inspiradas nos níveis da [Taxonomia de Bloom](https://cft.vanderbilt.edu/guides-sub-pages/blooms-taxonomy/).

| Nível | Descrição | [Tax. de Bloom](https://cft.vanderbilt.edu/guides-sub-pages/blooms-taxonomy/) |
| --- | --- | --- |
| F | Não entregue, entregue sem completar o `README.md`, entregue sem implementar a rotação (ver teoria), ou implementação usa bibliotecas prontas para implementar as transformações.  | Não fez |
| E | Entregue, mas o `README.md` não indica como instalar ou rodar o programa, ou a rotação não ocorre ao redor do centro do vídeo. | Entender (-) |
| D | Roda com alguns travamentos ou erros, ou o `README.md` não descreve o modelo matemático que foi aplicado, ou o `README.md` não corresponde ao modelo que foi implementado de fato, ou o programa pode ser facilmente acelerado aplicando a ideia de transformações compostas (ver a teoria). | Entender |
| C | Funciona sem travar e o `README.md` está completo e corresponde ao código ou o processo de rotação gera artefatos (pontos pretos) dentro da imagem (ver teoria). | Compreender (-) |
| C+ | O processo de rotação está correto, mas gera artefatos (distorções indesejadas) **fora** da imagem (por exemplo: frames sobrepostos, etc) | Compreender |
| B | O processo de rotação está correto e não gera artefatos | Aplicar |
| A | O programa obedece a todos os requisitos e o código tem uma correspondência imediata ao modelo matemático descrito no `README.md` | Analisar |
| A+ | O programa funciona perfeitamente e, em adição aos requisitos pedidos, permite interagir com a rotação usando teclado ou mouse, por exemplo controlando a velocidade angular através do mouse. | Avaliar |
| A++ | O programa funciona perfeitamente e, em adição aos requisitos anteriores, incorpora outra transformação controlável, como expansões ou cisalhamentos | Criar |
| A+++ | O programa funciona perfeitamente e, em adição aos requisitos anteriores, permite salvar o vídeo que está sendo feito. | Criar (+) |


# 